In [1]:
import sys
import math
import wandb
import torch
sys.path.append('..')
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from utils.data_modules.contrastive import EEGContrastiveDataModule
from models.trainers.contrastive import ContrastiveTrainerModel, PlottingCallback

/workspace/eeg-image-decoding/data/all-joined-1/eeg/epochs


In [2]:
dm = EEGContrastiveDataModule(
    input_channels=['Fp1', 'AF7', 'AF3', 'F1', 'F3', 'F5', 'F7', 'FT7', 'FC5', 'FC3', 'FC1', 'C1', 'C3', 'C5', 'T7', 'TP7', 'CP5', 'CP3', 'CP1', 'P1', 'P3', 'P5', 'P7', 'P9', 'PO7', 'PO3', 'O1', 'Iz', 'Oz', 'POz', 'Pz', 'CPz', 'Fpz', 'Fp2', 'AF8', 'AF4', 'AFz', 'Fz', 'F2', 'F4', 'F6', 'F8', 'FT8', 'FC6', 'FC4', 'FC2', 'FCz', 'Cz', 'C2', 'C4', 'C6', 'T8', 'TP8', 'CP6', 'CP4', 'CP2', 'P2', 'P4', 'P6', 'P8', 'P10', 'PO8', 'PO4', 'O2'],
    sfreq=250,
    montage='standard_1020',
    window_before_event_ms=50,
    window_after_event_ms=600,
    subject=1, 
    session=1, 
    batch_size=64, 
    num_workers=4
)

In [3]:
sample_data = dm.get_sample_info()
epochs = 200
subject = 1
session = 1
num_channels = sample_data['input']['num_channels']
timesteps = sample_data['input']['num_timesteps']
num_fine_labels = sample_data['output']['fine_labels_shape']

Creating Datasets...
Original dataframe size: 2887
Original dataframe size: 321
Original dataframe size: 631


In [4]:
lightning_model = ContrastiveTrainerModel(num_channels, timesteps, num_fine_labels)
    
checkpoint_callback = ModelCheckpoint(
    dirpath="/workspace/eeg-image-decoding/code/models/check_points/contrastive_encoder",
    filename=f"subj{subject}_session{session}_{{epoch}}",
    every_n_epochs=5,
    save_top_k=-1
)

plotting_callback = PlottingCallback()

logger = pl.loggers.WandbLogger(project="contrastive_eeg_net")

trainer = pl.Trainer(
    max_epochs=epochs,
    callbacks=[checkpoint_callback, plotting_callback],
    logger=logger,
    accelerator='gpu' if torch.cuda.is_available() else 'cpu',
    devices=1
)

trainer.fit(lightning_model, dm)

if logger and isinstance(logger, pl.loggers.WandbLogger):
    logger.finalize('success')
    wandb.finish()

/venv/main/lib/python3.10/site-packages/braindecode/models/base.py:23: UserWarning: EEGNetv4: 'in_chans' is depreciated. Use 'n_chans' instead.
  warnings.warn(
/venv/main/lib/python3.10/site-packages/braindecode/models/base.py:23: UserWarning: EEGNetv4: 'n_classes' is depreciated. Use 'n_outputs' instead.
  warnings.warn(
/venv/main/lib/python3.10/site-packages/braindecode/models/base.py:23: UserWarning: EEGNetv4: 'input_window_samples' is depreciated. Use 'n_times' instead.
  warnings.warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA H100 NVL') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
wandb: ERROR Failed to detect 

Creating Datasets...
Original dataframe size: 2887
Original dataframe size: 321


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name                  | Type                      | Params | Mode 
-----------------------------------------------------------------------------
0  | encoder               | EEGEncoderDefiner         | 859 K  | train
1  | train_loss            | MeanMetric                | 0      | train
2  | train_accuracy        | MultilabelAccuracy        | 0      | train
3  | val_loss              | MeanMetric                | 0      | train
4  | val_accuracy          | MultilabelAccuracy        | 0      | train
5  | v2_acc                | MultilabelAccuracy        | 0      | train
6  | v4_acc                | MultilabelAccuracy        | 0      | train
7  | v10_acc               | MultilabelAccuracy        | 0      | train
8  | v80_acc               | MultilabelAccuracy        | 0      | train
9  | top5_acc              | MeanMetric                | 0      | train
10 | test_precision        | MultilabelPrecision       | 0      | train
11 | test_recal

Original dataframe size: 631


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=200` reached.


No test predictions available for confusion matrix visualization


wandb: ERROR The nbformat package was not found. It is required to save notebook history.


epoch,▁▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇███
top5_acc,▄▇▅▅▃▄▂▃▂▃▁▅▆▅▄▄▅▄█▄▆▄▂▇▅▃▃▂▃▄▆▄▇▅▄▅▅▄▄▃
train_accuracy,▇████▇▇▇▇▇▆▆▅▅▅▄▃▃▃▃▃▂▂▂▂▂▁▂▂▂▁▁▁▁▁▂▂▂▁▂
train_loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
trainer/global_step,▁▁▁▁▂▂▃▃▃▃▃▃▄▄▄▄▄▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆█████
v10_acc,▇████▇▅▅▆▄▄▄▅▄▃▃▃▃▃▃▂▂▂▁▂▁▂▁▁▁▁▁▂▁▁▁▁▂▁▂
v2_acc,▇█▅▄▂▁▂▂▂▂▄▄▄▄▄▃▄▅▅▅▅▅▆▅▆▅▅▆▆▅▆▆▆▆▆▆▆▆▆▆
v4_acc,██▁▂▁▃▃▂▃▄▄▄▄▄▅▅▆▅▅▆▅▅▆▆▆▅▄▅▆▆▆▇▅▇▆▆▆▆▆▇
v80_acc,▁▅▇█▇█▇▇▇▇▆▅▅▅▅▆▅▅▄▄▄▄▄▄▄▃▄▄▄▃▃▃▃▃▃▃▃▃▃▃
val_accuracy,▁▂█▇██▇▇▆▅▅▆▅▅▄▃▄▄▄▃▄▄▃▃▃▃▃▃▂▂▃▃▂▃▂▃▃▂▃▃
val_loss,▁▁▅▆▇███████▇▇▇▇▇▇▇▇▇▇▇▇▇▇▆▆▆▆▆▆▆▆▆▆▆▆▆▆


In [5]:
results = []
for epoch in range(epochs):
    if epoch < len(plotting_callback.train_losses):
        epoch_results = {
            "epoch": epoch + 1,
            "train_loss": plotting_callback.train_losses[epoch],
            "train_accuracy": plotting_callback.train_accuracies[epoch],
            "test_loss": plotting_callback.test_losses[epoch],
            "test_accuracy": plotting_callback.test_accuracies[epoch],
            "v2_acc": plotting_callback.v2_accs[epoch],
            "v4_acc": plotting_callback.v4_accs[epoch],
            "v10_acc": plotting_callback.v10_accs[epoch],
            "v80_acc": plotting_callback.v80_accs[epoch]
        }
        results.append(epoch_results)

results

[{'epoch': 1,
  'train_loss': 0,
  'train_accuracy': 0,
  'test_loss': 0,
  'test_accuracy': 0,
  'v2_acc': 0,
  'v4_acc': 0,
  'v10_acc': 0,
  'v80_acc': 0},
 {'epoch': 2,
  'train_loss': 4.414199352264404,
  'train_accuracy': 0.5976359844207764,
  'test_loss': 0,
  'test_accuracy': 0,
  'v2_acc': 0,
  'v4_acc': 0,
  'v10_acc': 0,
  'v80_acc': 0},
 {'epoch': 3,
  'train_loss': 3.697460889816284,
  'train_accuracy': 0.5900675654411316,
  'test_loss': 3.465386390686035,
  'test_accuracy': 0.09754672646522522,
  'v2_acc': 0.8395638465881348,
  'v4_acc': 0.6066977977752686,
  'v10_acc': 0.3065420389175415,
  'v80_acc': 0.09754672646522522},
 {'epoch': 4,
  'train_loss': 3.1454596519470215,
  'train_accuracy': 0.6295332908630371,
  'test_loss': 3.4659945964813232,
  'test_accuracy': 0.03492990881204605,
  'v2_acc': 0.8831775784492493,
  'v4_acc': 0.6152647733688354,
  'v10_acc': 0.2781931459903717,
  'v80_acc': 0.03492990881204605},
 {'epoch': 5,
  'train_loss': 2.5300121307373047,
  'trai